In [1]:
import os
import json
import glob
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import plotly.graph_objs as go

from PIL import Image, ImageOps
from skimage import io
from skimage.color import rgba2rgb, rgb2xyz
from tqdm import tqdm
from dataclasses import dataclass
from math import floor, ceil
import random

# Train data generation
import collections
import csv
from pathlib import Path
from typing import List, Tuple, Any

import time
import re
from sklearn import preprocessing
import lightgbm as lgb

import multiprocessing
from multiprocessing import Pool

pd.set_option("display.max_columns", 100)

In [3]:
# Check out each file. Content, images

root_path = "/home/jupyter/input/"
train_paths = glob.glob(root_path + "train" + "/*/*/*")
test_paths = glob.glob(root_path + "test" + "/*")
metafiles = glob.glob(root_path + "metadata" + "/*")

print("No. Files in Train: {:,}".format(len(train_paths)), "\n" +
      "No. Files in Test: {:,}".format(len(test_paths)), "\n" +
      "No. of metadata files: {:,}".format(len(metafiles)))

No. Files in Train: 26,925 
No. Files in Test: 626 
No. of metadata files: 204


In [51]:
# Get submission file
sub_df = pd.read_csv("/home/jupyter/input/sample_submission.csv")
sub_df[["site", "file", "timestamp"]] = sub_df["site_path_timestamp"].apply(lambda x: pd.Series(x.split("_")))
sub_df = sub_df.drop(columns=["floor", "x", "y"])
display(sub_df.head())
print(len(sub_df))

,site_path_timestamp,site,file,timestamp
0,5a0546857ecc773753327266_046cfa46be49fc1083481...,5a0546857ecc773753327266,046cfa46be49fc10834815c6,0000000000009
1,5a0546857ecc773753327266_046cfa46be49fc1083481...,5a0546857ecc773753327266,046cfa46be49fc10834815c6,0000000009017
2,5a0546857ecc773753327266_046cfa46be49fc1083481...,5a0546857ecc773753327266,046cfa46be49fc10834815c6,0000000015326
3,5a0546857ecc773753327266_046cfa46be49fc1083481...,5a0546857ecc773753327266,046cfa46be49fc10834815c6,0000000018763
4,5a0546857ecc773753327266_046cfa46be49fc1083481...,5a0546857ecc773753327266,046cfa46be49fc10834815c6,0000000022328


10133


In [68]:
# 200 train paths come out with ~1000 examples, so multiply train examples by 5 to extract similar no. of examples
# test_num = train_num * 5
# test_num = 100
test_num = len(sub_df)
# test_num = round(len(sub_df) / 2)

# set timestamp cut
time_stamp_cut = 1000

print(test_num)
print(len(sub_df.iloc[:test_num, :]))

10133
10133


In [57]:
# using github repo in kaggle kernels
# https://www.kaggle.com/getting-started/71642
# !cp -r /home/jupyter/input/indoorlocationcompetition20master/indoor-location-competition-20-master/* ./

In [58]:
import compute_f
import io_f
import visualize_f
import main
from io_f import read_data_file

In [59]:
# Try working out step_positions for 1 trace file
from compute_f import compute_step_positions, compute_steps, \
compute_headings, compute_stride_length, compute_step_heading, compute_rel_positions, split_ts_seq

# Feature candidate
# You can't get the waypoint in test, so use acce and ahrs data to calculate relative positions
def calc_rel_positions(acce_datas, ahrs_datas):
    step_timestamps, step_indexs, step_acce_max_mins = compute_steps(acce_datas)
    headings = compute_headings(ahrs_datas)
    stride_lengths = compute_stride_length(step_acce_max_mins)
    step_headings = compute_step_heading(step_timestamps, headings)
    rel_positions = compute_rel_positions(stride_lengths, step_headings)
    # only use del if we don't need timestamps
    # rel_positions_del = np.delete(rel_positions, 0, 1)
    return rel_positions

# Feature candidate
# Modify extract_magnetic_strength from github for one magnetic data point
def extract_one_magn_strength(magn_datas):
    d = np.array(magn_datas[2:5])
    return np.mean(np.sqrt(np.sum(d ** 2, axis=0)))

In [60]:
# Methods for preprocessing train data: Timestamp handling
def find_diff_ts(ts, data):
    data_ts = data[0]
    diff_ts = int(data_ts) - int(ts)
    return diff_ts

def find_start_ts(path):
    with open(path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    for line_data in lines:
        line_data = line_data.strip()
        m = re.search(r"(?<=startTime.)(.*)", line_data)
        start_ts = m.groups(0)
        if m:
            return (start_ts[0])

def find_smallest_diff(t, data):
    data_ts = data[:, [0]]
    diff = []
    for ts in data_ts:
        diff.append(abs(int(t) - int(ts)))
    closest_index = np.argmin(diff) # if multiple records have the same value..?
    return data[closest_index]

In [61]:
# Method for preprocessing train data: splitting acce/ahrs/gyro/magn
def split_axis(data, start_ts):
    data_ts = data[0]
    diff_ts = int(data[0]) - int(start_ts)
    x_axis = data[1]
    y_axis = data[2]
    z_axis = data[3]
    try:
        accuracy = data[4]
    except IndexError:
        accuracy = np.nan
    return [data_ts, diff_ts, x_axis, y_axis, z_axis, accuracy]

# Method for preprocessing train data: splitting wifi
def split_wifi(data, start_ts):
    data_ts = data[0]
    diff_ts = int(data[0]) - int(start_ts)
    ssid = data[1]
    bssid = data[2]
    rssi = data[3]
    if len(data) > 5:
        freq = data[4]
        last_seen_ts = data[5]
    else:
        freq = np.nan
        last_seen_ts = data[-1]
    return [data_ts, diff_ts, ssid, bssid, rssi, freq, last_seen_ts]

# Method for preprocessing train data: splitting ibeacon
def split_beacon(data, start_ts):
    data_ts = data[0]
    diff_ts = int(data[0]) - int(start_ts)
    ssid = data[1]
    rssi = data[2]
    return [data_ts, diff_ts, ssid, rssi]

# Method for preprocessing train data: calc rel pos
def split_rel_pos(data, start_ts):
    data_ts = data[0]
    diff_ts = int(data[0]) - int(start_ts)
    x_axis = data[1]
    y_axis = data[2]
    return [data_ts, diff_ts, x_axis, y_axis]

In [62]:
floor_map = {"B3":-3,"B2":-2,"B1":-1,"F1":0,"1F":0,"F2":1,"2F":1,"F3":2,"3F":2,"F4":3,"4F":3,
             "F5":4,"5F":4,"F6":5,"6F":5,"F7":6,"7F":6,"F8":7,"8F": 7,"F9":8,"9F":8,"F10":9,
             "B":0,"BF":1,"BM":2, "G":0, "M":0, "P1":0,"P2":1, "LG2":-2,"LG1":-1,"LG":0,"LM":0,
             "L1":1,"L2":2,"L3":3,"L4":4,"L5":5,"L6":6,"L7":7,"L8":8,"L9":9,"L10":10,"L11":11}

In [63]:
# def extract_test_data(df):
#     test_rows = []
#     for index, row in df.iterrows():
#         test_site = row["site"]
#         file_name = row["file"]
#         test_ts = row["timestamp"]

#         test_path = root_path + "test/" + file_name + ".txt" # get test_path from file name
#         start_ts = find_start_ts(test_path)
#         diff_start_ts = int(test_ts) - int(start_ts)
#         path_datas = read_data_file(test_path)
#         acce = path_datas.acce
#         ahrs = path_datas.ahrs
#         magn = path_datas.magn
#         wifi = path_datas.wifi

#         # extract data for each timestamp of waypoints
#         res = []
#         acce_closest = split_axis(find_smallest_diff(test_ts, acce))
#         ahrs_closest = split_axis(find_smallest_diff(test_ts, ahrs))
#         magn_closest = split_axis(find_smallest_diff(test_ts, magn))
#         wifi_closest = split_wifi(find_smallest_diff(test_ts, wifi))
#         test_rows.append([test_site, file_name, np.nan, np.nan, test_ts, np.nan, np.nan, start_ts, diff_start_ts] + \
#                           acce_closest + ahrs_closest + magn_closest + wifi_closest + \
#                          [acce_closest[0], ahrs_closest[0], magn_closest[0], wifi_closest[0]])
#     return test_rows

In [64]:
# Generate test data
def extract_test_data(df):
    try:
        test_rows = []
        for index, row in df.iterrows():
            row_id = row["site_path_timestamp"]
            test_site = row["site"]
            file_name = row["file"]
            test_ts = row["timestamp"]
            test_path = root_path + "test/" + file_name + ".txt" # get test_path from file name

            # extract data for each timestamp of waypoints
            start_ts = find_start_ts(test_path)
            path_datas = read_data_file(test_path)
            acce = path_datas.acce
            ahrs = path_datas.ahrs
            magn = path_datas.magn
            gyro = path_datas.gyro
            acce_uncali = path_datas.acce_uncali
            magn_uncali = path_datas.magn_uncali
            gyro_uncali = path_datas.gyro_uncali
            wifi = path_datas.wifi
            wps = path_datas.waypoint
            ibeacon = path_datas.ibeacon
            rel_positions = calc_rel_positions(acce, ahrs)

            # Changed from: just extracting wps time stamps -> take all acce uncalib timestamps
            # ts = np.unique(wps[:, [0]])
            ts = np.unique(acce_uncali[:, [0]]) # take uncalibrated access, as sometimes access has less data

            # extract data for each timestamp of waypoints
            for i, t in enumerate(ts):
                # wp_closest = find_smallest_diff(t, wps)
                # closest_wp_ts = wp_closest[0]
                diff_ts_wp_ts = int(t) - int(test_ts)
                # time_stamp_cut = 2000, only the records within 2 sec of test timestamp
                if abs(diff_ts_wp_ts) < time_stamp_cut:
                    # flag to indicate how close the data point is to the wps
                    within_500ms = True if abs(diff_ts_wp_ts) <= 500 else False
                    within_1000ms = True if abs(diff_ts_wp_ts) <= 1000 else False
                    # x = wp_closest[1]
                    # y = wp_closest[2]
                    diff_start_ts = int(t) - int(start_ts)
                    diff_start_wp_ts = int(test_ts)
                    acce_closest = split_axis(find_smallest_diff(t, acce), start_ts)
                    ahrs_closest = split_axis(find_smallest_diff(t, ahrs), start_ts)
                    magn_closest = split_axis(find_smallest_diff(t, magn), start_ts)
                    magn_closest.append(extract_one_magn_strength(magn_closest)) # append magnetic strength only for the magn data
                    gyro_closest = split_axis(find_smallest_diff(t, gyro), start_ts)
                    acce_u_closest = split_axis(find_smallest_diff(t, acce_uncali), start_ts)
                    magn_u_closest = split_axis(find_smallest_diff(t, magn_uncali), start_ts)
                    gyro_u_closest = split_axis(find_smallest_diff(t, gyro_uncali), start_ts)
                    wifi_closest = split_wifi(find_smallest_diff(t, wifi), start_ts)
                    beacon_closest = split_beacon(find_smallest_diff(t, ibeacon), start_ts)
                    rel_pos = split_rel_pos(find_smallest_diff(t, rel_positions), start_ts)

                    # train append example
                    # res.append([int(t), start_ts, diff_start_ts, x, y, int(closest_wp_ts), diff_start_wp_ts, diff_ts_wp_ts, within_500ms, within_1000ms] + \
                    #            acce_closest + ahrs_closest + magn_closest + gyro_closest + \
                    #            acce_u_closest + magn_u_closest + gyro_u_closest + \
                    #            wifi_closest + beacon_closest + rel_pos
                    #           )
                    test_rows.append([row_id, test_site, file_name, np.nan, np.nan, \
                                      t, start_ts, diff_start_ts, np.nan, np.nan, test_ts, diff_start_wp_ts, diff_ts_wp_ts, within_500ms, within_1000ms] + \
                                      acce_closest + ahrs_closest + magn_closest + gyro_closest + \
                                      acce_u_closest + magn_u_closest + gyro_u_closest + \
                                      wifi_closest + beacon_closest + rel_pos
                                    )
                else:
                    continue

        return test_rows
    except:
        print("extract_test_data error")

In [65]:
# can read_data_file method read test data
print(test_paths[0])
test_path = test_paths[0]
read_data_file(test_path)
find_start_ts(test_path)

/home/jupyter/input/test/32e88857b1938abaebce9a76.txt


'0000000000000'

In [48]:
# try generating test data
# kaggle notebook -> 48.8 sec
# here -> 28.9 sec
start = time.time()
test_rows = extract_test_data(sub_df.iloc[:5, :])
print(f"time to process: ", time.time() - start)

time to process:  28.970062971115112


In [49]:
test_df = pd.DataFrame(test_rows)
display(test_df.head())
print(len(test_rows[0]))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72
0,5a0546857ecc773753327266_046cfa46be49fc1083481...,5a0546857ecc773753327266,046cfa46be49fc10834815c6,NaN,NaN,136.0,0000000000000,136,NaN,NaN,0000000000009,9,127,True,True,136.0,136,0.798813,4.300720,7.810059,NaN,136.0,136,0.247101,0.104201,0.474897,NaN,136.0,136,30.561829,-1.228333,-38.301086,NaN,49.015379,136.0,136,-0.039139,-0.507996,-0.148392,NaN,136.0,136,0.578552,4.353989,8.195526,NaN,136.0,136,34.687805,6.938171,-377.32544,NaN,136.0,136,-0.077835,-0.334671,-0.166565,NaN,0000000002340,2340,da39a3ee5e6b4b0d3255bfef95601890afd80709,eebf5db207eec2f3e041f92153d789270f346821,-45,NaN,1578474544726,0000000000110,110,d9c573b719a17da4836208fc436f87b5ca1aa877_b6589...,-91,1144.0,1144,-0.425353,0.24869
1,5a0546857ecc773753327266_046cfa46be49fc1083481...,5a0546857ecc773753327266,046cfa46be49fc10834815c6,NaN,NaN,156.0,0000000000000,156,NaN,NaN,0000000000009,9,147,True,True,156.0,156,0.026688,4.911835,7.244446,NaN,156.0,156,0.247101,0.104201,0.474897,NaN,156.0,156,29.173279,-1.922607,-36.950684,NaN,47.118252,156.0,156,-0.084946,-0.447800,-0.207520,NaN,156.0,156,0.524673,4.493454,7.438370,NaN,156.0,156,33.299255,6.243896,-375.97504,NaN,156.0,156,-0.049072,-0.504059,-0.155380,NaN,0000000002340,2340,da39a3ee5e6b4b0d3255bfef95601890afd80709,eebf5db207eec2f3e041f92153d789270f346821,-45,NaN,1578474544726,0000000000110,110,d9c573b719a17da4836208fc436f87b5ca1aa877_b6589...,-91,1144.0,1144,-0.425353,0.24869
2,5a0546857ecc773753327266_046cfa46be49fc1083481...,5a0546857ecc773753327266,046cfa46be49fc10834815c6,NaN,NaN,176.0,0000000000000,176,NaN,NaN,0000000000009,9,167,True,True,176.0,176,-0.418640,5.354172,7.295914,NaN,176.0,176,0.254368,0.111510,0.452041,NaN,176.0,176,27.786255,-0.535583,-37.625122,NaN,46.776197,176.0,176,-0.110519,-0.327423,-0.163315,NaN,176.0,176,-0.269592,5.217697,7.327042,NaN,176.0,176,31.912231,7.630920,-376.64948,NaN,176.0,176,-0.094879,-0.443863,-0.214508,NaN,0000000002340,2340,da39a3ee5e6b4b0d3255bfef95601890afd80709,eebf5db207eec2f3e041f92153d789270f346821,-45,NaN,1578474544726,0000000000216,216,d9c573b719a17da4836208fc436f87b5ca1aa877_b6589...,-71,1144.0,1144,-0.425353,0.24869
3,5a0546857ecc773753327266_046cfa46be49fc1083481...,5a0546857ecc773753327266,046cfa46be49fc10834815c6,NaN,NaN,196.0,0000000000000,196,NaN,NaN,0000000000009,9,187,True,True,196.0,196,-0.540146,5.456512,7.113358,NaN,196.0,196,0.254368,0.111510,0.452041,NaN,196.0,196,27.786255,-0.535583,-36.950684,NaN,46.235439,196.0,196,-0.267639,-0.278961,-0.078094,NaN,196.0,196,-0.555710,5.455322,7.103775,NaN,196.0,196,31.912231,7.630920,-375.97504,NaN,196.0,196,-0.120453,-0.323486,-0.170303,NaN,0000000002340,2340,da39a3ee5e6b4b0d3255bfef95601890afd80709,eebf5db207eec2f3e041f92153d789270f346821,-45,NaN,1578474544726,0000000000216,216,d9c573b719a17da4836208fc436f87b5ca1aa877_b6589...,-71,1144.0,1144,-0.425353,0.24869
4,5a0546857ecc773753327266_046cfa46be49fc1083481...,5a0546857ecc773753327266,046cfa46be49fc10834815c6,NaN,NaN,216.0,0000000000000,216,NaN,NaN,0000000000009,9,207,True,True,216.0,216,-0.078064,5.031540,7.386291,NaN,216.0,216,0.256831,0.115168,0.441099,NaN,216.0,216,27.786255,-0.535583,-38.975525,NaN,47.869138,216.0,216,-0.461517,-0.306656,-0.005112,NaN,216.0,216,-0.302521,5.271561,7.274963,NaN,216.0,216,31.912231,7.630920,-377.99988,NaN,216.0,216,-0.277573,-0.275024,-0.085083,NaN,0000000002340,2340,da39a3ee5e6b4b0d3255bfef95601890afd80709,eebf5db207eec2f3e041f92153d789270f346821,-45,NaN,1578474544726,0000000000216,216,d9c573b719a17da4836208fc436f87b5ca1aa877_b6589...,-71,1144.0,1144,-0.425353,0.24869


73


In [69]:
# Pool for test data
# kaggle notebook -> 144.7 sec for 30 paths
# here -> 26.6 sec for 30 paths at time_stamp_cut = 2000
# here -> 89.5 sec for 100 paths at time_stamp_cut = 2000
# here -> 51.4 sec for 100 paths at time_stamp_cut = 1000
# -> 5100 sec for ~10,000 examples -> 85 min or so
# sub_df length = 10133

def apply_pool_to_df(df, f, pool, num_cores):
    result = pool.map(f, [d for d in np.array_split(df, num_cores)])
    pool.close()
    return result

num_cores = multiprocessing.cpu_count()
pool = Pool(num_cores)
print("cores: ", num_cores)

start = time.time()
res = apply_pool_to_df(sub_df.iloc[:test_num, :], extract_test_data, pool, num_cores)
print(f"time to process {test_num} examples of sub_df", time.time() - start)

cores:  16
extract_test_data error
extract_test_data error
extract_test_data error
extract_test_data error
extract_test_data error
extract_test_data error
extract_test_data error
extract_test_data error
extract_test_data error
extract_test_data error
extract_test_data error
extract_test_data error
time to process 10133 examples of sub_df 6686.089239835739


In [70]:
col_names = ["site_path_timestamp", "site_id", "file_id", "floor_converted", "floor", \
             "ts", "start_ts", "diff_start_ts", "x", "y", \
             "closest_wp_ts", "diff_start_wp_ts", "diff_ts_wp_ts", "within_500ms", "within_1000ms", \
             "acce_ts", "diff_acce_ts", "acce_x", "acce_y", "acce_z", "acce_acc", \
             "ahrs_ts", "diff_ahrs_ts", "ahrs_x", "ahrs_y", "ahrs_z", "ahrs_acc", \
             "magn_ts", "diff_magn_ts", "magn_x", "magn_y", "magn_z", "magn_acc", "magn_strength",\
             "gyro_ts", "diff_gyro_ts", "gyro_x", "gyro_y", "gyro_z", "gyro_acc", \
             "acce_u_ts", "diff_acce_u_ts", "acce_u_x", "acce_u_y", "acce_u_z", "acce_u_acc", \
             "magn_u_ts", "diff_magn_u_ts", "magn_u_x", "magn_u_y", "magn_u_z", "magn_u_acc", \
             "gyro_u_ts", "diff_gyro_u_ts", "gyro_u_x", "gyro_u_y", "gyro_u_z", "gyro_u_acc", \
             "wifi_ts", "diff_wifi_ts", "wifi_ssid", "wifi_bssid", "wifi_rssi", "wifi_freq", "wifi_last_seen_ts", \
             "beacon_ts", "diff_beacon_ts", "beacon_ssid", "beacon_rssi", \
             "rel_ts", "diff_rel_ts", "rel_x", "rel_y"
            ]

df_test = pd.DataFrame(res[0], columns=col_names)
for r in res[1:]:
    df = pd.DataFrame(r, columns=col_names)
    df_test = df_test.append(df)
df_test = df_test.set_index("site_path_timestamp")

# process 1000 records -> 173.9 sec -> all test records are ~10,000 -> 1740 sec (~29min)
print("test_path count", len(test_paths[:test_num]))
print("length of df made", len(df_test))
display(df_test.head(10))

test_path count 626
length of df made 242441


,site_id,file_id,floor_converted,floor,ts,start_ts,diff_start_ts,x,y,closest_wp_ts,diff_start_wp_ts,diff_ts_wp_ts,within_500ms,within_1000ms,acce_ts,diff_acce_ts,acce_x,acce_y,acce_z,acce_acc,ahrs_ts,diff_ahrs_ts,ahrs_x,ahrs_y,ahrs_z,ahrs_acc,magn_ts,diff_magn_ts,magn_x,magn_y,magn_z,magn_acc,magn_strength,gyro_ts,diff_gyro_ts,gyro_x,gyro_y,gyro_z,gyro_acc,acce_u_ts,diff_acce_u_ts,acce_u_x,acce_u_y,acce_u_z,acce_u_acc,magn_u_ts,diff_magn_u_ts,magn_u_x,magn_u_y,magn_u_z,magn_u_acc,gyro_u_ts,diff_gyro_u_ts,gyro_u_x,gyro_u_y,gyro_u_z,gyro_u_acc,wifi_ts,diff_wifi_ts,wifi_ssid,wifi_bssid,wifi_rssi,wifi_freq,wifi_last_seen_ts,beacon_ts,diff_beacon_ts,beacon_ssid,beacon_rssi,rel_ts,diff_rel_ts,rel_x,rel_y
site_path_timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000000009,5a0546857ecc773753327266,046cfa46be49fc10834815c6,NaN,NaN,136.0,0000000000000,136,NaN,NaN,0000000000009,9,127,True,True,136.0,136,0.798813,4.300720,7.810059,NaN,136.0,136,0.247101,0.104201,0.474897,NaN,136.0,136,30.561829,-1.228333,-38.301086,NaN,49.015379,136.0,136,-0.039139,-0.507996,-0.148392,NaN,136.0,136,0.578552,4.353989,8.195526,NaN,136.0,136,34.687805,6.938171,-377.32544,NaN,136.0,136,-0.077835,-0.334671,-0.166565,NaN,0000000002340,2340,da39a3ee5e6b4b0d3255bfef95601890afd80709,eebf5db207eec2f3e041f92153d789270f346821,-45,NaN,1578474544726,0000000000110,110,d9c573b719a17da4836208fc436f87b5ca1aa877_b6589...,-91,1144.0,1144,-0.425353,0.24869
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000000009,5a0546857ecc773753327266,046cfa46be49fc10834815c6,NaN,NaN,156.0,0000000000000,156,NaN,NaN,0000000000009,9,147,True,True,156.0,156,0.026688,4.911835,7.244446,NaN,156.0,156,0.247101,0.104201,0.474897,NaN,156.0,156,29.173279,-1.922607,-36.950684,NaN,47.118252,156.0,156,-0.084946,-0.447800,-0.207520,NaN,156.0,156,0.524673,4.493454,7.438370,NaN,156.0,156,33.299255,6.243896,-375.97504,NaN,156.0,156,-0.049072,-0.504059,-0.155380,NaN,0000000002340,2340,da39a3ee5e6b4b0d3255bfef95601890afd80709,eebf5db207eec2f3e041f92153d789270f346821,-45,NaN,1578474544726,0000000000110,110,d9c573b719a17da4836208fc436f87b5ca1aa877_b6589...,-91,1144.0,1144,-0.425353,0.24869
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000000009,5a0546857ecc773753327266,046cfa46be49fc10834815c6,NaN,NaN,176.0,0000000000000,176,NaN,NaN,0000000000009,9,167,True,True,176.0,176,-0.418640,5.354172,7.295914,NaN,176.0,176,0.254368,0.111510,0.452041,NaN,176.0,176,27.786255,-0.535583,-37.625122,NaN,46.776197,176.0,176,-0.110519,-0.327423,-0.163315,NaN,176.0,176,-0.269592,5.217697,7.327042,NaN,176.0,176,31.912231,7.630920,-376.64948,NaN,176.0,176,-0.094879,-0.443863,-0.214508,NaN,0000000002340,2340,da39a3ee5e6b4b0d3255bfef95601890afd80709,eebf5db207eec2f3e041f92153d789270f346821,-45,NaN,1578474544726,0000000000216,216,d9c573b719a17da4836208fc436f87b5ca1aa877_b6589...,-71,1144.0,1144,-0.425353,0.24869
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000000009,5a0546857ecc773753327266,046cfa46be49fc10834815c6,NaN,NaN,196.0,0000000000000,196,NaN,NaN,0000000000009,9,187,True,True,196.0,196,-0.540146,5.456512,7.113358,NaN,196.0,196,0.254368,0.111510,0.452041,NaN,196.0,196,27.786255,-0.535583,-36.950684,NaN,46.235439,196.0,196,-0.267639,-0.278961,-0.078094,NaN,196.0,196,-0.555710,5.455322,7.103775,NaN,196.0,196,31.912231,7.630920,-375.97504,NaN,196.0,196,-0.120453,-0.323486,-0.170303,NaN,0000000002340,2340,da39a3ee5e6b4b0d3255bfef95601890afd80709,eebf5db207eec2f3e041f92153d789270f346821,-45,NaN,1578474544726,0000000000216,216,d9c573b719a17da4836208fc436f87b5ca1aa877_b6589...,-71,1144.0,1144,-0.425353,0.24869
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000000009,5a0546857ecc773753327266,046cfa46be49fc10834815c6,NaN,NaN,216.0,0000000000000,216,NaN,NaN,0000000000009,9,207,True,True,216.0,216,-0.078064,5.031540,7.386291,NaN,216.0,216,0.256831,0.115168,0.441099,NaN,216.0,216,27.786255,-0.535583,-38.975525,NaN,47.86913

In [71]:
# LabelEncode site_id, file_id, floor_converted, ssid, bssid
def col_encode(df, cols):
    for col in cols:
        le = preprocessing.LabelEncoder()
        df["%s_le"%col] = le.fit_transform(df[col])

col_enc = ["site_id", "file_id", "floor", "wifi_ssid", "wifi_bssid", "beacon_ssid"]
col_encode(df_test, col_enc)

# convert data types of certain columns
def convert_dtypes(df, col_list, dtype):
    for col in col_list:
        df[col] = df[col].astype(dtype)

convert_dtypes(df_test, ["ts", "start_ts", "diff_start_ts", \
                         "closest_wp_ts", "diff_start_wp_ts", "diff_ts_wp_ts",\
                         "acce_ts", "diff_acce_ts", "ahrs_ts", "diff_ahrs_ts", \
                         "magn_ts", "diff_magn_ts", "gyro_ts", "diff_gyro_ts", \
                         "acce_u_ts", "diff_acce_u_ts", "magn_u_ts", "diff_magn_u_ts", \
                         "gyro_u_ts", "diff_gyro_u_ts", \
                         "wifi_ts", "diff_wifi_ts", "wifi_rssi", "wifi_freq", "wifi_last_seen_ts", \
                         "beacon_ts", "diff_beacon_ts", "beacon_rssi", "rel_ts", "diff_rel_ts"
                        ], float)

# convert ts and wifi_last_see_ts to dates
for df in [df_test]:
    for col in ["ts", "wifi_last_seen_ts"]:
        df["%s_date"%col] = pd.to_datetime(df[col],unit="ms")
        df["%s_day"%col] = df["%s_date"%col].dt.floor("d")
        df["%s_hour"%col] = df["%s_date"%col].dt.floor("h")
        df["%s_minute"%col] = df["%s_date"%col].values.astype("<M8[m]")

# Check
display(df_test.head())

,site_id,file_id,floor_converted,floor,ts,start_ts,diff_start_ts,x,y,closest_wp_ts,diff_start_wp_ts,diff_ts_wp_ts,within_500ms,within_1000ms,acce_ts,diff_acce_ts,acce_x,acce_y,acce_z,acce_acc,ahrs_ts,diff_ahrs_ts,ahrs_x,ahrs_y,ahrs_z,ahrs_acc,magn_ts,diff_magn_ts,magn_x,magn_y,magn_z,magn_acc,magn_strength,gyro_ts,diff_gyro_ts,gyro_x,gyro_y,gyro_z,gyro_acc,acce_u_ts,diff_acce_u_ts,acce_u_x,acce_u_y,acce_u_z,acce_u_acc,magn_u_ts,diff_magn_u_ts,magn_u_x,magn_u_y,magn_u_z,magn_u_acc,gyro_u_ts,diff_gyro_u_ts,gyro_u_x,gyro_u_y,gyro_u_z,gyro_u_acc,wifi_ts,diff_wifi_ts,wifi_ssid,wifi_bssid,wifi_rssi,wifi_freq,wifi_last_seen_ts,beacon_ts,diff_beacon_ts,beacon_ssid,beacon_rssi,rel_ts,diff_rel_ts,rel_x,rel_y,site_id_le,file_id_le,floor_le,wifi_ssid_le,wifi_bssid_le,beacon_ssid_le,ts_date,ts_day,ts_hour,ts_minute,wifi_last_seen_ts_date,wifi_last_seen_ts_day,wifi_last_seen_ts_hour,wifi_last_seen_ts_minute
site_path_timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000000009,5a0546857ecc773753327266,046cfa46be49fc10834815c6,NaN,NaN,136.0,0.0,136.0,NaN,NaN,9.0,9.0,127.0,True,True,136.0,136.0,0.798813,4.300720,7.810059,NaN,136.0,136.0,0.247101,0.104201,0.474897,NaN,136.0,136.0,30.561829,-1.228333,-38.301086,NaN,49.015379,136.0,136.0,-0.039139,-0.507996,-0.148392,NaN,136.0,136.0,0.578552,4.353989,8.195526,NaN,136.0,136.0,34.687805,6.938171,-377.32544,NaN,136.0,136.0,-0.077835,-0.334671,-0.166565,NaN,2340.0,2340.0,da39a3ee5e6b4b0d3255bfef95601890afd80709,eebf5db207eec2f3e041f92153d789270f346821,-45.0,NaN,1.578475e+12,110.0,110.0,d9c573b719a17da4836208fc436f87b5ca1aa877_b6589...,-91.0,1144.0,1144.0,-0.425353,0.24869,0,1,0,541,1222,1034,1970-01-01 00:00:00.136,1970-01-01,1970-01-01,1970-01-01,2020-01-08 09:09:04.726000128,2020-01-08,2020-01-08 09:00:00,2020-01-08 09:09:00
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000000009,5a0546857ecc773753327266,046cfa46be49fc10834815c6,NaN,NaN,156.0,0.0,156.0,NaN,NaN,9.0,9.0,147.0,True,True,156.0,156.0,0.026688,4.911835,7.244446,NaN,156.0,156.0,0.247101,0.104201,0.474897,NaN,156.0,156.0,29.173279,-1.922607,-36.950684,NaN,47.118252,156.0,156.0,-0.084946,-0.447800,-0.207520,NaN,156.0,156.0,0.524673,4.493454,7.438370,NaN,156.0,156.0,33.299255,6.243896,-375.97504,NaN,156.0,156.0,-0.049072,-0.504059,-0.155380,NaN,2340.0,2340.0,da39a3ee5e6b4b0d3255bfef95601890afd80709,eebf5db207eec2f3e041f92153d789270f346821,-45.0,NaN,1.578475e+12,110.0,110.0,d9c573b719a17da4836208fc436f87b5ca1aa877_b6589...,-91.0,1144.0,1144.0,-0.425353,0.24869,0,1,0,541,1222,1034,1970-01-01 00:00:00.156,1970-01-01,1970-01-01,1970-01-01,2020-01-08 09:09:04.726000128,2020-01-08,2020-01-08 09:00:00,2020-01-08 09:09:00
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000000009,5a0546857ecc773753327266,046cfa46be49fc10834815c6,NaN,NaN,176.0,0.0,176.0,NaN,NaN,9.0,9.0,167.0,True,True,176.0,176.0,-0.418640,5.354172,7.295914,NaN,176.0,176.0,0.254368,0.111510,0.452041,NaN,176.0,176.0,27.786255,-0.535583,-37.625122,NaN,46.776197,176.0,176.0,-0.110519,-0.327423,-0.163315,NaN,176.0,176.0,-0.269592,5.217697,7.327042,NaN,176.0,176.0,31.912231,7.630920,-376.64948,NaN,176.0,176.0,-0.094879,-0.443863,-0.214508,NaN,2340.0,2340.0,da39a3ee5e6b4b0d3255bfef95601890afd80709,eebf5db207eec2f3e041f92153d789270f346821,-45.0,NaN,1.578475e+12,216.0,216.0,d9c573b719a17da4836208fc436f87b5ca1aa877_b6589...,-71.0,1144.0,1144.0,-0.425353,0.24869,0,1,0,541,1222,1034,1970-01-01 00:00:00.176,1970-01-01,1970-01-01,1970-01-01,2020-01-08 09:09:04.726000128,2020-01-08,2020-01-08 09:00:00,2020-01-08 09:09:00
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000000009,5a0546857ecc773753327266,046cfa46be49fc10834815c6,NaN,NaN,196.0,0.0,196.0,NaN,NaN,9.0,9.0,187.0,True,True,196.0,196.0,-0.540146,5.456512,7.113358,NaN,196.0,196.0,0.254368,0.111510,0.452041,NaN,196.0,196.0,27.786255,-0.535583,-36.950684,NaN,46.235439,196.0,196.0,-0.267639,-0.278961,-0.078094,NaN,196.0,196.0,

In [72]:
# Save the file in parquet
# https://www.kaggle.com/pedrocouto39/fast-reading-w-pickle-feather-parquet-jay
# https://www.kaggle.com/prmohanty/python-how-to-save-and-load-ml-models
import pickle

# Saving train data
test_file_name = "indoor_test.pkl"
with open(test_file_name, "wb") as file:
    pickle.dump(df_test, file)

# Save them to output
df_test.to_csv('df_test.csv')